In [1]:
# Clone github for this mini project which is presented an Udacity course
!git clone https://github.com/udacity/ud120-projects

Cloning into 'ud120-projects'...
remote: Enumerating objects: 5173, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 5173 (delta 38), reused 72 (delta 31), pack-reused 5079
Receiving objects: 100% (5173/5173), 19.67 MiB | 22.74 MiB/s, done.
Resolving deltas: 100% (4432/4432), done.


In [9]:
import joblib
import numpy

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectPercentile, f_classif

def preprocess(words_file = "ud120-projects/tools/word_data.pkl", authors_file="ud120-projects/tools/email_authors.pkl"):
    """ 
        this function takes a pre-made list of email texts (by default word_data.pkl)
        and the corresponding authors (by default email_authors.pkl) and performs
        a number of preprocessing steps:
            -- splits into training/testing sets (10% testing)
            -- vectorizes into tfidf matrix
            -- selects/keeps most helpful features

        after this, the feaures and labels are put into numpy arrays, which play nice with sklearn functions

        4 objects are returned:
            -- training/testing features
            -- training/testing labels

    """

    ### the words (features) and authors (labels), already largely preprocessed
    ### this preprocessing will be repeated in the text learning mini-project
    authors_file_handler = open(authors_file, "rb")
    authors = joblib.load(authors_file_handler)


    words_file_handler = open(words_file, "rb")
    word_data = joblib.load(words_file_handler)

    ### test_size is the percentage of events assigned to the test set
    ### (remainder go into training)
    features_train, features_test, labels_train, labels_test = train_test_split(word_data, authors, test_size=0.1, random_state=42)

    ### text vectorization--go from strings to lists of numbers
    vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words='english')
    features_train_transformed = vectorizer.fit_transform(features_train)
    features_test_transformed  = vectorizer.transform(features_test)

    ### feature selection, because text is super high dimensional and 
    ### can be really computationally chewy as a result
    selector = SelectPercentile(f_classif, percentile=10)
    selector.fit(features_train_transformed, labels_train)
    features_train_transformed = selector.transform(features_train_transformed).toarray()
    features_test_transformed  = selector.transform(features_test_transformed).toarray()

    ### info on the data
    print("No. of Chris training emails : ", sum(labels_train))
    print("No. of Sara training emails : ", len(labels_train)-sum(labels_train))
    
    return features_train_transformed, features_test_transformed, labels_train, labels_test

In [22]:
""" 
    This is the code to accompany the Lesson 2 (SVM) mini-project.
    Use a SVM to identify emails from the Enron corpus by their authors:    
    Sara has label 0
    Chris has label 1
"""
    
import sys
from time import time

### features_train and features_test are the features for the training
### and testing datasets, respectively
### labels_train and labels_test are the corresponding item labels
features_train, features_test, labels_train, labels_test = preprocess()


#########################################################
### your code goes here ###
from sklearn import svm

# speed- up the learning process to optimize model
# features_train = features_train[:int(len(features_train)/100)]
# labels_train = labels_train[:int(len(labels_train)/100)]

# t0 = time()
clf_rbf = svm.SVC(kernel='rbf',C=10000)
# clf_linear = svm.SVC(kernel='linear')
clf_rbf.fit(features_train,labels_train)
# print("Training Time:", round(time()-t0, 3), "s")


# t1 = time()
pred = clf_rbf.predict(features_test)
# print("Prediction Time:", round(time()-t1, 3), "s")

accuracy = clf_rbf.score(features_test, labels_test)

print(accuracy)

# Make some predictions
# print(pred[[10,26,50]])

# How many of predictions are Chris(1) ?
print(sum(pred))
#########################################################

No. of Chris training emails :  7936
No. of Sara training emails :  7884
0.9960182025028441
866
